![Piggy bank](piggy_bank.jpg)

Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two-year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to ensure it conforms to the specific structure and data types that they specify so that they can then use the cleaned data you provide to set up a PostgreSQL database, which will store this campaign's data and allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split the data, saving three final csv files. Specifically, the three files should have the names and contents as outlined below:

+ Clean bank_marketing.csv and store as three DataFrames called `client`, `campaign`, and `economics`, each containing the columns outlined in the notebook and formatted to the data types listed.

+ Save the three DataFrames to csv files, without an index, as `client.csv`, `campaign.csv`, and `economics.csv` respectively.

## `client.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `age` | `integer` | Client's age in years | N/A |
| `job` | `object` | Client's type of job | Change `"."` to `"_"` |
| `marital` | `object` | Client's marital status | N/A |
| `education` | `object` | Client's level of education | Change `"."` to `"_"` and `"unknown"` to `np.NaN` |
| `credit_default` | `bool` | Whether the client's credit is in default | Convert to boolean data type |
| `mortgage` | `bool` | Whether the client has an existing mortgage (housing loan) | Convert to boolean data type |

<br>

## `campaign.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign | N/A |
| `contact_duration` | `integer` | Last contact duration in seconds | N/A |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign | N/A |
| `previous_outcome` | `bool` | Outcome of the previous campaign | Convert to boolean data type |
| `campaign_outcome` | `bool` | Outcome of the current campaign | Convert to boolean data type |
| `last_contact_date` | `datetime` | Last date the client was contacted | Create from a combination of `day`, `month`, and a newly created `year` column (which should have a value of `2022`); <br> **Format =** `"YYYY-MM-DD"` |

<br>

## `economics.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) | N/A |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three-month rate (daily indicator) | N/A |

In [66]:
import pandas as pd
import numpy as np

# Start coding here...
file = "bank_marketing.csv"
bank_marketing_df = pd.read_csv(file)
bank_marketing_df.head(2)

,client_id,age,job,marital,education,credit_default,mortgage,month,day,contact_duration,number_contacts,previous_campaign_contacts,previous_outcome,cons_price_idx,euribor_three_months,campaign_outcome
0,0,56,housemaid,married,basic.4y,no,no,may,13,261,1,0,nonexistent,93.994,4.857,no
1,1,57,services,married,high.school,unknown,no,may,19,149,1,0,nonexistent,93.994,4.857,no


In [67]:
# Check the data types
bank_marketing_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 16 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   client_id                   41188 non-null  int64  
 1   age                         41188 non-null  int64  
 2   job                         41188 non-null  object 
 3   marital                     41188 non-null  object 
 4   education                   41188 non-null  object 
 5   credit_default              41188 non-null  object 
 6   mortgage                    41188 non-null  object 
 7   month                       41188 non-null  object 
 8   day                         41188 non-null  int64  
 9   contact_duration            41188 non-null  int64  
 10  number_contacts             41188 non-null  int64  
 11  previous_campaign_contacts  41188 non-null  int64  
 12  previous_outcome            41188 non-null  object 
 13  cons_price_idx              411

In [68]:
# Task desc: Change(job col) "." to "_"
bank_marketing_df['job'] = bank_marketing_df['job'].str.replace('.', '_')

# Task desc: Change(education col) "." to "_" and "unknown" to np.NaN
bank_marketing_df['education'] = bank_marketing_df['education'].str.replace('.', '_')
bank_marketing_df.loc[bank_marketing_df['education'] == 'unknown', 'education' ] = np.NaN

# Task desc: Convert(credit_default) to boolean data type
bank_marketing_df['credit_default'] = bank_marketing_df['credit_default'].astype('bool')

# Task desc: Convert(mortgage) to boolean data type
bank_marketing_df['mortgage'] = bank_marketing_df['mortgage'].astype('bool')

# Task desc: Convert(previous_outcome) to boolean data type
bank_marketing_df['previous_outcome'] = bank_marketing_df['previous_outcome'].astype('bool')

# Task desc: Convert(campaign_outcome) to boolean data 
bank_marketing_df['campaign_outcome'] = bank_marketing_df['campaign_outcome'].astype('bool')

# Create from a combination of day, month, and a newly created year column (which should have a value of 2022); Format = "YYYY-MM-DD"

# Task desc: create a new column 'year' with constant value 2022
bank_marketing_df['year'] = 2022
# map months(str) to their appropriate integer values
# first make sure the month values are in consistent string format
bank_marketing_df['month'] = bank_marketing_df['month'].str.lower()
month_map = {
    'january': 1, 'february': 2, 'march': 3, 'april': 4,
    'may': 5, 'june': 6, 'july': 7, 'august': 8,
    'september': 9, 'october': 10, 'november': 11, 'december': 12
}

# Convert the month column to numeric values
bank_marketing_df['month'] = bank_marketing_df['month'].map(month_map)

# Create a datetime column by combining 'year', 'month', and 'day'
bank_marketing_df['last_contact_date'] = pd.to_datetime(bank_marketing_df[['year', 'month', 'day']], format='%Y-%m-%d')


In [69]:
# Check if the dataframe is now cleaned
bank_marketing_df.head()

,client_id,age,job,marital,education,credit_default,mortgage,month,day,contact_duration,number_contacts,previous_campaign_contacts,previous_outcome,cons_price_idx,euribor_three_months,campaign_outcome,year,last_contact_date
0,0,56,housemaid,married,basic_4y,True,True,5.0,13,261,1,0,True,93.994,4.857,True,2022,2022-05-13
1,1,57,services,married,high_school,True,True,5.0,19,149,1,0,True,93.994,4.857,True,2022,2022-05-19
2,2,37,services,married,high_school,True,True,5.0,23,226,1,0,True,93.994,4.857,True,2022,2022-05-23
3,3,40,admin_,married,basic_6y,True,True,5.0,27,151,1,0,True,93.994,4.857,True,2022,2022-05-27
4,4,56,services,married,high_school,True,True,5.0,3,307,1,0,True,93.994,4.857,True,2022,2022-05-03


In [70]:
# Chek if data types are corrected
bank_marketing_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 18 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   client_id                   41188 non-null  int64         
 1   age                         41188 non-null  int64         
 2   job                         41188 non-null  object        
 3   marital                     41188 non-null  object        
 4   education                   39457 non-null  object        
 5   credit_default              41188 non-null  bool          
 6   mortgage                    41188 non-null  bool          
 7   month                       13769 non-null  float64       
 8   day                         41188 non-null  int64         
 9   contact_duration            41188 non-null  int64         
 10  number_contacts             41188 non-null  int64         
 11  previous_campaign_contacts  41188 non-null  int64     

In [73]:
# create the client columns subset
client = bank_marketing_df[['client_id', 'age', 'job',  'marital', 'education', 'credit_default', 'mortgage' ]]

# create the campaign columns subset
campaign = bank_marketing_df[['client_id', 'contact_duration', 'number_contacts', 'previous_campaign_contacts', 'previous_outcome', 'campaign_outcome', 'last_contact_date' ]]
# create the economics columns subset
economics = bank_marketing_df[['client_id', 'cons_price_idx', 'euribor_three_months' ]]
# Save DataFrames to CSV files without an index
client.to_csv('client.csv', index=False)
campaign.to_csv('campaign.csv', index=False)
economics.to_csv('economics.csv', index=False)
